In [1]:
import pandas as pd
import numpy as np
import pathlib
import datetime
from rich import print
#import xarray as xr

In [2]:
pth = pathlib.Path.home() / pathlib.Path("als/cvsbdata", follow_links=True)

In [3]:
pth.resolve()

PosixPath('/Users/kdavis10/als/cvsbdata')

In [29]:
try:
    del cl
except NameError:
    pass

cl = pd.read_csv("/Users/kdavis10/iCloud_Documents/Documents/Personal/Records/csvb/data/Combined Ledger.csv",
                    header=3, parse_dates=["Date"])
cl = cl.set_index("LedgerID")
cl["Amount"] = cl["Amount"].astype(float)

In [5]:
df=cl
np.expand_dims(df["Amount"].to_numpy(),0) == -1 * np.expand_dims(df["Amount"].to_numpy(),0).T
np.abs(np.expand_dims(df.Date,0) - np.expand_dims(df.Date,0).T).astype('timedelta64[D]') < np.timedelta64(5, "D")

array([[ True,  True, False, ..., False, False, False],
       [ True,  True,  True, ..., False, False, False],
       [False,  True,  True, ..., False, False, False],
       ...,
       [False, False, False, ...,  True,  True,  True],
       [False, False, False, ...,  True,  True,  True],
       [False, False, False, ...,  True,  True,  True]])

In [8]:
def identify_possible_transfers(df):
    # Compare the transposes to find where Amount == -Amount and the date is similar.
    amount_matches = np.expand_dims(df["Amount"].to_numpy(),0) == -1 * np.expand_dims(df["Amount"].to_numpy(),0).T
    date_matches = np.abs(np.expand_dims(df.Date,0) - np.expand_dims(df.Date,0).T).astype('timedelta64[D]') < np.timedelta64(5, "D")
    matchix = np.where(np.logical_and(amount_matches, date_matches))

    # The following could be one line, but it would be impossible to remember how it works.
    lIDpairs = zip(cl.index[matchix[0]], cl.index[matchix[1]]) # Make a list of id locations.
    lIDpairs = set([tuple(np.sort(lIDpair)) for lIDpair in lIDpairs if lIDpair[0] != lIDpair[1]]) # Sort pairs so that i,j is indentical to j,i and run through a `set` to remove duplicates.
    lIDpairs = list(lIDpairs)

    # Create a dataframe with the values for the multindex.
    pairdf = pd.DataFrame(np.array(lIDpairs), columns=["left", "right"])
    pairdf.index.name = "matchID"
    
    left = pd.DataFrame(pairdf["left"])
    left["side"] = "left"
    right = pd.DataFrame(pairdf["right"])
    right["side"] = "right"

    udf = pd.concat([left.rename(columns={"left": "LedgerID"}),
                     right.rename(columns={"right": "LedgerID"})
                    ]).sort_index().reset_index()
    
    # Below: Select table rows for each side of match.
    # reset the index so that it LedgerID becomes a column.
    # Set the index to a multiindex created from the indexing frame.
    clmi = cl.loc[udf["LedgerID"]].reset_index().set_index(pd.MultiIndex.from_frame(udf[["matchID", "side"]]))
    clmi["is_match"] = np.NaN # Create a column to hold human verification.
    
    return clmi



In [11]:
clmi = identify_possible_transfers(cl)
clmi.to_csv("possible_matches_UNVERIFIED.csv")

In [138]:
clmi.to_csv("clmi.csv")

In [21]:
cl[np.abs(cl["Amount"]) == 35]

,Ledger,Date,Description,Amount,Account
LedgerID,,,,,
1CD58EB6,Apple:Credit Card,2022-09-18,TRADE COFFEE CO 268 SUMMER ST. 6TH FLOOR BOSTO...,-35.0,Groceries
AD5E8BD2,Apple:Credit Card,2022-10-16,TRADE COFFEE CO 268 SUMMER ST. 6TH FLOOR BOSTO...,-35.0,Subscriptions
5E85A256,Apple:Credit Card,2022-11-13,TRADE COFFEE CO 268 SUMMER ST. 6TH FLOOR BOSTO...,-35.0,Groceries
0C607EDA,Apple:Credit Card,2022-12-11,TRADE COFFEE CO 268 SUMMER ST. 6TH FLOOR BOSTO...,-35.0,Groceries
F9968E47,Apple:Credit Card,2023-01-08,TRADE COFFEE CO 268 SUMMER ST. 6TH FLOOR BOSTO...,-35.0,Groceries
44EAA926,Apple:Credit Card,2023-02-05,TRADE COFFEE CO 268 SUMMER ST. 6TH FLOOR BOSTO...,-35.0,Groceries
0AF2A20E,Apple:Credit Card,2023-02-28,TRADE COFFEE CO 268 SUMMER ST. 6TH FLOOR BOSTO...,-35.0,Groceries
3FEE0FC6,Apple:Credit Card,2023-03-27,TRADE COFFEE CO 268 SUMMER ST. 6TH FLOOR BOSTO...,-35.0,Groceries
B26F0962,Apple:Credit Card,2023-04-24,TRADE COFFEE CO 268 SUMMER ST. 6TH FLOOR BOSTO...,-35.0,Groceries


In [200]:
xf = pd.read_csv("/Users/kdavis10/iCloud_Documents/Documents/Personal/Records/csvb/data/products/confirmed_xfers_12-25-2023.csv", header=0, parse_dates=["Date"])
xf = xf[xf["is_xfer"] == True].drop(columns=["is_xfer"])

xfdash = xf.loc[xf["side"] == "left", "LedgerID"] + pd.Series("-", index=xf.loc[xf["side"] == "left", "LedgerID"].index)
xfr = xf.loc[xf["side"] == "right", "LedgerID"]
xfr.index = xfdash.index
joinedleft = xfdash + xfr
joinedright = xfdash + xfr
joinedright.index = xf.loc[xf["side"] == "right", "LedgerID"].index
xf["TransferID"] = pd.concat([joinedleft, joinedright])
xf.set_index(pd.MultiIndex.from_frame(xf[["TransferID", "side"]])).drop(columns=["TransferID", "side"])
xf.to_csv("/Users/kdavis10/iCloud_Documents/Documents/Personal/Records/csvb/data/products/Transfers.csv")